In [ ]:
import time
from selenium import webdriver
import pandas as pd
import threading
import random
import sqlite3
RANGE =10
THEADING =3
def InitChrome():
    options = webdriver.ChromeOptions()

    options.add_argument('lang=zh_CN.UTF-8')
#    options.add_argument('user-agent="Mozilla/5.0 (iPod; U; CPU iPhone OS 2_1 like Mac OS X; ja-jp) AppleWebKit/525.18.1 (KHTML, like Gecko) Version/3.1.1 Mobile/5F137 Safari/525.20"')
    prefs = {
        'profile.default_content_setting_values': {
        'images': 2
        }
    }
    options.add_argument('--headless') 
#    options.add_argument('--proxy-server='+ip)
    options.add_experimental_option('prefs',prefs)
    browser = webdriver.Chrome(chrome_options=options)
    return browser

def InitSQL():
    conn = sqlite3.connect('TmallInformation.db')
    conn.execute('create table content (Id int primary key,Brands text,Content text);')
    print('Init SQL Success')
    
def SaveSQL(data):
    with sqlite3.connect('TmallInformation.db') as con:
        data.to_sql(name='brands',con = con ,if_exists='append')
    print('save success')
    
def getInformation(data,number):
    print('Th-',number,end = '\t')
    content=[]

    for i in data.Brands:
        if '/' in i[0]:
            i = re.split('/',i[0])[1]
        if '（' in i[0]:
            i = re.split('（',i[0])[0]
        print(i)
        browser = InitChrome()
        browser.get('https://www.tmall.com/')
        browser.find_element_by_xpath('//*[@id="mq"]').clear()
        # 传入数据
        browser.find_element_by_xpath('//*[@id="mq"]').send_keys(i)
        time.sleep(random.randint(3,5))
        browser.find_element_by_xpath('//*[@id="mallSearch"]/form/fieldset/div/button').click()
        try:
            content.append(browser.find_element_by_xpath('//*[@id="J_ItemList"]').text)
        except:
            content.append('LOST')
        browser.close()
    me=pd.DataFrame(content,index= data.index,columns=['Content'])
    result = pd.concat((data,me),axis=1)
    SaveSQL(result)
    


def InputData():
    with sqlite3.connect('TmallInformation.db') as con:
        data= pd.read_sql_query('select * from BrandsAfFE;',con=con,index_col='brandsIndex')
    return data

def ChromeThread(listRange):
    temp = []
    for i in range(len(listRange)//THEADING):
        print('Wheel',i*THEADING*RANGE,'-',(i+1)*THEADING*RANGE)
        temp = listRange[i*THEADING:(i+1)*THEADING]
        count = 0
        pool = []
        for packgae in temp:
            count+=1
            pool.append(threading.Thread(target = getInformation, args = (packgae, count)))
        for j in pool:
            j.start()
        for j in pool:
            j.join()
        print('Wheel ' ,i ,' OVER')

def main():
#    InitSQL()
    ToSidata = InputData()
    with sqlite3.connect('TmallInformation.db') as con:
        alreadyData = pd.read_sql_query('select * from content;',con=con,index_col='Id')
    result=ToSidata[~ToSidata.index.isin(alreadyData.index)]
    listRange = []
    for i in range(len(result)//RANGE):
        listRange.append(result.iloc[i*RANGE:(i+1)*RANGE])
    ChromeThread(listRange)
        
if __name__ =='__main__':
    main()

In [14]:
def InputData():
    with sqlite3.connect('TmallInformation.db') as con:
        data= pd.read_sql_query('select * from BrandsAfFE;',con=con,index_col='brandsIndex')
    return data
InputData()

,text,Brands
brandsIndex,,
0,进口纯牛奶,None
1,进口酸奶,德亚
1,进口酸奶,卓德
1,进口酸奶,阿贝多
1,进口酸奶,SUKI/多美鲜
1,进口酸奶,安佳
1,进口酸奶,甘蒂牧场
1,进口酸奶,Arla/爱氏晨曦
1,进口酸奶,菲仕利


In [6]:
import pandas as pd
import sqlite3
con = sqlite3.connect('TmallInformation.db')
sql = 'select Brands from brands;'
data = pd.read_sql(sql = sql,con =con)